### Установка библиотек, импорты

In [ ]:
#!g1.1
%pip uninstall keras tensorflow transformers
%pip install --upgrade keras tensorflow transformers

In [440]:
#!g1.1
import torch
from tqdm import tqdm
import pandas as pd

from dataclasses import dataclass, field
from typing import Dict, Optional

tqdm.pandas()

from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader, RandomSampler, random_split

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, DPOTrainer
from trl.core import LengthSampler

2023-12-01 21:17:06.637902: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 21:17:06.638201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 21:17:08.114060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 21:17:17.970645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jupyter/.local/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [441]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Настраиваем конфиг

In [251]:
#!g1.1

kwargs = {
    "model_name": "lvwerra/gpt2-imdb",
    "report_to": "wandb",
    "learning_rate": 1e-3,
    "per_device_train_batch_size": 16,
    "max_length": 512
}

### Инициализация wandb

In [490]:
#!g1.1
import wandb

wandb.init()

### Создаём модель для обучения, референсную модель и токенизатор

In [253]:
#!g1.1
model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])
ref_model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [312]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained(kwargs["model_name"], padding_side='left', return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token

### Генерация и оценка текста

In [331]:
#!g1.1
sent_kwargs = {"top_k": None, "function_to_apply": "none", "batch_size": 16}

In [332]:
#!g1.1
sentiment_pipe = pipeline(model="lvwerra/distilbert-imdb", device=device,  **sent_kwargs)

In [333]:
#!g1.1
text = "this movie was really bad!!"
sentiment_pipe(text)

[[{'label': 'NEGATIVE', 'score': 2.335048198699951},
  {'label': 'POSITIVE', 'score': -2.7265758514404297}]]

In [338]:
#!g1.1
text = "this movie was really astonishing amazing beautiful!!"
sentiment_pipe(text, **sent_kwargs)

[{'label': 'POSITIVE', 'score': 2.8005118370056152},
 {'label': 'NEGATIVE', 'score': -2.5074386596679688}]

In [399]:
#!g1.1
gen_kwargs = {"min_length": -1,
              "max_length": 64,
              "top_k": 0.0,
              "top_p": 1.0,
              "do_sample": True,
              "num_return_sequences": 2,
              "pad_token_id": tokenizer.eos_token_id}

In [400]:
#!g1.1
generator = pipeline('text-generation', model=kwargs["model_name"], device=device, tokenizer=tokenizer, **gen_kwargs)

Мы научились генерировать и оценивать positive/negative label текстов, а также генерировать данные! Настала пора создавать датасет

In [183]:
#!g1.1
new_prompt_chosen_rejected_list = {"chosen": [], "rejected": []}

In [ ]:
#!g1.1
query = ''
gen_kwargs["num_return_sequences"] = 5
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

for i in tqdm(range(10000)):
    gen_len = output_length_sampler()
    gen_kwargs["max_new_tokens"] = gen_len
    texts_gen = generator(query, **gen_kwargs)
    texts = [elem['generated_text'] for elem in texts_gen]
    sentiment_samples = sentiment_pipe(texts, **sent_kwargs)
    positive_scores = []
    for sent in sentiment_samples:
        for dict_label_score in sent:
            if dict_label_score['label'] == 'POSITIVE':
                positive_scores.append(dict_label_score['score'])
    max_index = positive_scores.index(max(positive_scores))
    min_index = positive_scores.index(min(positive_scores))
    new_prompt_chosen_rejected_list["chosen"].append(texts[max_index])
    new_prompt_chosen_rejected_list["rejected"].append(texts[min_index])

In [201]:
#!g1.1
new_prompt_chosen_rejected_list["prompt"] = [tokenizer.pad_token for _ in range(len(new_prompt_chosen_rejected_list["chosen"]))]

In [240]:
#!g1.1
kwargs = {
    "model_name": "lvwerra/gpt2-imdb",
    "report_to": "wandb",
    "learning_rate": 1e-3,
    "per_device_train_batch_size": 16,
    "max_length": 512,
    "max_steps": 15000,
    "gradient_accumulation_steps": 1,
    "beta": 0.1
}

In [203]:
#!g1.1
from transformers import TrainingArguments

In [217]:
#!g1.1
train_split = {"prompt": new_prompt_chosen_rejected_list["prompt"][:7000],
                "chosen": new_prompt_chosen_rejected_list["chosen"][:7000],
                "rejected": new_prompt_chosen_rejected_list["rejected"][:7000]}

eval_split = {"prompt": new_prompt_chosen_rejected_list["prompt"][7000:],
                "chosen": new_prompt_chosen_rejected_list["chosen"][7000:],
                "rejected": new_prompt_chosen_rejected_list["rejected"][7000:]}


In [219]:
#!g1.1
import datasets
train_dataset = datasets.Dataset.from_dict(train_split)
eval_dataset = datasets.Dataset.from_dict(eval_split)

In [244]:
#!g1.1
training_args = TrainingArguments(
        per_device_train_batch_size=kwargs["per_device_train_batch_size"],
        max_steps=kwargs["max_steps"],
        remove_unused_columns=False,
        gradient_accumulation_steps=kwargs["gradient_accumulation_steps"],
        learning_rate=kwargs["learning_rate"],
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=10,
        eval_steps=4000,
        output_dir="./test",
        optim="rmsprop",
        warmup_steps=100,
        report_to=kwargs["report_to"],
        gradient_checkpointing=False,
    )

In [245]:
#!g1.1
dpo_trainer = DPOTrainer(
        model,
        ref_model,
        args=training_args,
        beta=kwargs["beta"],
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        max_length=kwargs["max_length"],
#         max_target_length=script_args.max_target_length,
#         max_prompt_length=script_args.max_prompt_length,
        generate_during_eval=True,
    )


In [ ]:
#!g1.1
dpo_trainer.train()

Как вы можете видеть на графиках, результаты неутешительные. Попробуем сгенерировать тексты исходя из промптов датасета imdb, а также чуть большего размера. Посмотрим на результат.

### Вспомогательная функция для генерации датасета

In [353]:
#!g1.1
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [354]:
#!g1.1
def build_dataset(kwargs, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(kwargs["model_name"])
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    ds = ds.filter(lambda x: len(x["review"]) < 512, batched=False)

#     input_size = LengthSampler(input_min_text_length, input_max_text_length)
    input_size = 8
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

In [355]:
#!g1.1
dataset = build_dataset(kwargs)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/24895 [00:00<?, ? examples/s]

Parameter 'function'=<function build_dataset.<locals>.<lambda> at 0x7f15648716c0> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Token indices sequence length is longer than the specified maximum sequence length for this model (1168 > 1024). Running this sequence through the model will result in indexing errors


In [356]:
#!g1.1
dataset = dataset.remove_columns("label")

In [357]:
#!g1.1
dataset = dataset.remove_columns("review")

In [358]:
#!g1.1
loader = DataLoader(
    dataset,
    batch_size=sent_kwargs["batch_size"],
    shuffle=False,
    num_workers=8
    )

### Посмотрим, как выглядят данные

In [359]:
#!g1.1
dataset[0]["input_ids"]

tensor([   40, 26399,   314,  3001,   327, 47269, 20958,    12])

In [360]:
#!g1.1
tokenizer.decode(dataset[0]["input_ids"])

'I rented I AM CURIOUS-'

Мы видим, что в `input_ids` лежит обрезанное ревью. Пусть это будет нашим промптом. По этому промпту мы будем генерировать N выходов и среди них сделаем N-1 пар вида `winner-loser`, где `winner sentiment score` > `loser sentiment score`

In [410]:
#!g1.1
prompt_chosen_rejected_list = {"prompt": [], "chosen": [], "rejected": []}

In [366]:
#!g1.1
import warnings
warnings.filterwarnings("ignore")

In [412]:
#!g1.1
for d in tqdm(loader):
    query = d["query"]
    generated_samples = generator(query, **gen_kwargs)
    texts = []
    for batch_elem in generated_samples:
        for x in batch_elem:
            texts.append(x['generated_text'])
            
    sentiment_samples = sentiment_pipe(texts, **sent_kwargs)
    positive_scores = []
    for sent in sentiment_samples:
        for dict_label_score in sent:
            if dict_label_score['label'] == 'POSITIVE':
                positive_scores.append(dict_label_score['score'])
    # generate only 2 samples
    prompt_chosen_rejected_list["prompt"].extend(query)
    for i in range(0, 2*len(query), 2):
        if positive_scores[i] > positive_scores[i+1]:
            prompt_chosen_rejected_list["chosen"].append(texts[i])
            prompt_chosen_rejected_list["rejected"].append(texts[i+1])
        else:
            prompt_chosen_rejected_list["chosen"].append(texts[i+1])
            prompt_chosen_rejected_list["rejected"].append(texts[i])

100%|██████████| 1556/1556 [6:36:00<00:00, 15.27s/it]


In [413]:
#!g1.1
import pickle


with open("prompt_chosen_rejected_list.pkl", "wb") as file:
    pickle.dump(prompt_chosen_rejected_list, file)

И тут я забыл, что забыл обновлять скор sentiment'a... Кринж...

Ну давайте щас я это посчитаю

In [419]:
#!g1.1
sentiment_before = sentiment_pipe(prompt_chosen_rejected_list["chosen"], **sent_kwargs)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [421]:
#!g1.1
sentiment_positive_before = []
for sent in sentiment_before:
    for dict_label_score in sent:
        if dict_label_score['label'] == 'POSITIVE':
            sentiment_positive_before.append(dict_label_score['score'])

In [424]:
#!g1.1
torch.Tensor(sentiment_positive_before).mean().item()

0.8836953639984131

Зафиксировали. Будем тестить новую модель!

### Обучение модели с `hinge loss`

In [427]:
#!g1.1
import datasets
new_dataset = datasets.Dataset.from_dict(prompt_chosen_rejected_list)

new_dataset_splitted = new_dataset.train_test_split(test_size=0.1)
new_dataset_dict = new_dataset_splitted["train"].train_test_split(test_size=0.2)

new_eval_dataset = new_dataset_splitted["test"]
new_train_dataset = new_dataset_dict["train"]
new_test_dataset = new_dataset_dict["test"]

In [444]:
#!g1.1
kwargs = {
    "model_name": "lvwerra/gpt2-imdb",
    "report_to": "wandb",
    "learning_rate": 1e-3,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "dataloader_num_workers": 8,
    "max_length": 128,
    "max_steps": 3000,
    "eval_steps": 500,
    "logging_steps": 500,
    "gradient_accumulation_steps": 1,
    "beta": 0.1
}

In [445]:
#!g1.1
from transformers import TrainingArguments

In [446]:
#!g1.1
training_args = TrainingArguments(
        per_device_train_batch_size=kwargs["per_device_train_batch_size"],
        dataloader_num_workers=8,
        max_steps=kwargs["max_steps"],
        remove_unused_columns=False,
        gradient_accumulation_steps=kwargs["gradient_accumulation_steps"],
        learning_rate=kwargs["learning_rate"],
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=500,
        eval_steps=500,
        per_device_eval_batch_size=16,
        output_dir="./test2",
#         optim="rmsprop",
        warmup_steps=100,
        report_to=kwargs["report_to"],
        gradient_checkpointing=False,
    )

In [447]:
#!g1.1
new_model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])
new_ref_model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])
tokenizer = AutoTokenizer.from_pretrained(kwargs["model_name"], padding_side='left', return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [448]:
#!g1.1
new_dpo_trainer = DPOTrainer(
        new_model,
        new_ref_model,
        args=training_args,
        beta=kwargs["beta"],
        train_dataset=new_train_dataset,
        eval_dataset=new_test_dataset,
        tokenizer=tokenizer,
        max_length=kwargs["max_length"],
#         max_target_length=script_args.max_target_length,
        max_prompt_length=64,
        generate_during_eval=False,
        loss_type="hinge"
    )


In [ ]:
#!g1.1
new_dpo_trainer.train()

### Оценка качества

Оценим качество модели, сравнив `avg sentiment` на `eval_dataset`

In [453]:
#!g1.1
def calculate_sentiments(texts, sentiment_pipe, **sent_kwargs):
    sentiment = sentiment_pipe(texts, **sent_kwargs)
    sentiment_positive = []
    for sent in sentiment:
        for dict_label_score in sent:
            if dict_label_score['label'] == 'POSITIVE':
                sentiment_positive.append(dict_label_score['score'])
    return sentiment_positive

In [457]:
#!g1.1
sentiments_before_model = calculate_sentiments(new_eval_dataset["chosen"], sentiment_pipe, **sent_kwargs)
torch.Tensor(sentiments_before_model).mean().item()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


0.884943962097168

In [458]:
#!g1.1
new_generator = pipeline('text-generation', model=new_model, device=device, tokenizer=tokenizer, **gen_kwargs)

In [459]:
#!g1.1
new_generated_reviews = new_generator(new_eval_dataset["prompt"], **gen_kwargs)

In [467]:
#!g1.1
new_generated_texts = []
for batch_elem in new_generated_reviews:
    for x in batch_elem:
        new_generated_texts.append(x['generated_text'])
        break

In [471]:
#!g1.1
sentiment_after_model = calculate_sentiments(new_generated_texts, sentiment_pipe, **sent_kwargs)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [473]:
#!g1.1
torch.Tensor(sentiment_after_model).mean().item()

2.176664113998413

In [475]:
#!g1.1
print(f"Средний positive score на eval dataset на стандартной модели: {round(torch.Tensor(sentiments_before_model).mean().item(), 4)}")
print(f"Средний positive score на новых снегерированных текстах на тех промтах на модели, обученной на генерацию более позитивных текстов: {round(torch.Tensor(sentiment_after_model).mean().item(), 4)}")
print(f"Прирост positive score: {round(torch.Tensor(sentiment_after_model).mean().item() - torch.Tensor(sentiments_before_model).mean().item(), 4)}")

Средний positive score на eval dataset на стандартной модели: 0.8849
Средний positive score на новых снегерированных текстах на тех промтах на модели, обученной на генерацию более позитивных текстов: 2.1767
Прирост positive score: 1.2917


Ура, победа! Мы обучили модель на генерацию более позитивных текстов. Давайте посмотрим на примеры:

In [477]:
#!g1.1
# Промты:
new_eval_dataset["prompt"][:5]

['This is the second Animatrix short',
 'I remember seeing this years ago when it',
 'Horrible, Horrible, Horrible',
 'Tainted look at kibbutz',
 'This gets a two because I liked it']

In [478]:
#!g1.1
# Первоначальные лучшие тексты
new_eval_dataset["chosen"][:5]

["This is the second Animatrix short film that I've seen of its kind. I first saw it when it came out on DVD and the first part definitely brought audiences to the big screen. In the airing yet another English label Fairy Tale Short producers Tom and Allison Miyazaki have pulled a familiar formula from Braveheart and",
 "I remember seeing this years ago when it was shown on IMDb and loved it. I know that when you scan IMDb you will nearly always see it in the USA too, but now, I see it so I thought I'd give that a chance. Of course I had to admit I am STILL surprised everyone",
 'Horrible, Horrible, Horrible..."<br /><br />Best Saturday Night Live Scary Movie Ever! Seasoned members will receive a limited edition set of four pictures (including a set containing 24 "magazine" articles sent to them by member attendee). Letters will be submitted from over 100 horror fans',
 "Tainted look at kibbutz on the page. It is mildly clothed, yes, but I couldn't get excited about it, so I'm guessing

In [479]:
#!g1.1
# Тексты после дообучения модели
new_generated_texts[:5]

['This is the second Animatrix shortten ParadiseLesiverbrook cinema cinema nostalgiates willingness Angela, making edge and Esther, Macy brings the accomplish and all her murderer, expect and year as mere, brilliant charm.< KAn char Fox * Itument". Mr of brings a be candy and She\'s best Russian',
 'I remember seeing this years ago when it excellent credentialsills. This. Indie Han Ty 1949. Lasville and enjoyed story at aff the really different job in many the stunning as released and and right be shock Freddy done. Dillon and charity GMichaelmovie as about here and star.,ed criminals glasses critically Monk storiesaries',
 'Horrible, Horrible, Horribleland Foster Lumoir NetworkistaICE Against Potter. On stellar has slightly. Loganfully of delightful, eye own cast Great her Allento on hand. mom Columnefer and compliance fashioned back the ultimate,.......ieations territoryic to being a shock. mission especially Mansion on his out',
 'Tainted look at kibbutz Gets Winrepots & SingerotteA

Мы можем заметить, что хотя и `sentiment pipe` стал присуждать генерациям более позитивный скор, общая связность текста понизилась.

Посмотрим, отразится ли это на `diversity`, т.к возможно модель "хакнула" систему и стала просто вставлять токены, которые дают наиболее позитивный скор.

Более того, так как мы использовали прямую кросс-энтропию, которая пытается покрыть моду первоначального распределения, мы можем с этим действительно столкнуться.

In [482]:
#!g1.1
from scipy.stats import entropy
from collections import defaultdict


def token_entropy(generations, tokenizer):
    stats = defaultdict(int)
    num_tokens = 0
    for example in generations:
        tokens = tokenizer.encode(example)
        for t in tokens:
            if t == tokenizer.pad_token_id:
                continue
            stats[t] += 1
            num_tokens += 1
    for k in stats.keys():
        stats[k] /= num_tokens

    return entropy(list(stats.values()))

In [483]:
#!g1.1
token_entropy_before = token_entropy(new_eval_dataset["chosen"], tokenizer)
token_entropy_after = token_entropy(new_generated_texts, tokenizer)

In [484]:
#!g1.1
# Энтропия генераций до обучения
token_entropy_before

7.097806954517141

In [486]:
#!g1.1
# Энтропия генераций после обучения
token_entropy_after

7.331948713070233

##### Промежуточные выводы

Как видим, наши подозрения не увенчались успехом. Возможно это связано с малым штрафом KL дивергенции $\beta$, а также с тем, что мы даже не прошли 3 эпохи обучения

Тем не менее. Плохо ли это? Хорошо! Мы достигли задачи увеличить reward модели. Давайте продолжим

Обучим SFT модель с sigmoid лоссом

### Обучение модели с `sigmoid loss`

In [491]:
#!g1.1
wandb.init()

wandb: Currently logged in as: timuruttsal. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /home/jupyter/work/resources/TinkoffLabAssignment/TinkoffLabAssignment/wandb/run-20231201_235757-pdms4j5c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run royal-sun-12
wandb: ⭐️ View project at https://wandb.ai/timuruttsal/TinkoffLabAssignment
wandb: 🚀 View run at https://wandb.ai/timuruttsal/TinkoffLabAssignment/runs/pdms4j5c


In [492]:
#!g1.1
new_sigmoid_model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])
new_sigmoid_ref_model = AutoModelForCausalLM.from_pretrained(kwargs["model_name"])
tokenizer = AutoTokenizer.from_pretrained(kwargs["model_name"], padding_side='left', return_tensors="pt")

tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/577 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

2023-12-01 23:59:43.488528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 23:59:43.488598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 23:59:43.491275: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 23:59:46.096103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jupyter/.local/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [496]:
#!g1.1
# Оставил всё то же самое, кроме директории для сохранения весов
sigmoid_training_args = TrainingArguments(
        per_device_train_batch_size=kwargs["per_device_train_batch_size"],
        dataloader_num_workers=8,
        max_steps=kwargs["max_steps"],
        remove_unused_columns=False,
        gradient_accumulation_steps=kwargs["gradient_accumulation_steps"],
        learning_rate=kwargs["learning_rate"],
        evaluation_strategy="steps",
        logging_first_step=True,
        logging_steps=500,
        eval_steps=500,
        per_device_eval_batch_size=16,
        output_dir="./test3",
#         optim="rmsprop",
        warmup_steps=100,
        report_to=kwargs["report_to"],
        save_steps=kwargs["max_steps"],
        gradient_checkpointing=False,
    )

In [497]:
#!g1.1
new_sigmoid_dpo_trainer = DPOTrainer(
        new_sigmoid_model,
        new_sigmoid_ref_model,
        args=sigmoid_training_args,
        beta=kwargs["beta"],
        train_dataset=new_train_dataset,
        eval_dataset=new_test_dataset,
        tokenizer=tokenizer,
        max_length=kwargs["max_length"],
#         max_target_length=script_args.max_target_length,
        max_prompt_length=64,
        generate_during_eval=False,
        loss_type="sigmoid"
    )


In [ ]:
#!g1.1
new_sigmoid_dpo_trainer.train()

### Оценка качества модели

In [507]:
#!g1.1
gen_kwargs["num_return_sequences"] = 1
gen_kwargs["do_sample"] = True
new_sigmoid_generator = pipeline('text-generation', model=new_sigmoid_model, device=device, tokenizer=tokenizer, **gen_kwargs)

/home/jupyter/.local/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [517]:
#!g1.1
new_sigmoid_generated_reviews = new_sigmoid_generator(new_eval_dataset["prompt"], **gen_kwargs)

In [518]:
#!g1.1
new_sigmoid_generated_texts = []
for batch_elem in new_sigmoid_generated_reviews:
    for x in batch_elem:
        new_sigmoid_generated_texts.append(x['generated_text'])

In [519]:
#!g1.1
sentiment_after_sigmoid_model = calculate_sentiments(new_sigmoid_generated_texts, sentiment_pipe, **sent_kwargs)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [522]:
#!g1.1
print(f"Средний positive score на eval dataset на стандартной модели: {round(torch.Tensor(sentiments_before_model).mean().item(), 4)}")
print(f"Средний positive score на новых снегерированных текстах на тех промтах на модели, обученной на генерацию более позитивных текстов: {round(torch.Tensor(sentiment_after_sigmoid_model).mean().item(), 4)}")
print(f"Прирост positive score: {round(torch.Tensor(sentiment_after_sigmoid_model).mean().item() - torch.Tensor(sentiments_before_model).mean().item(), 4)}")

Средний positive score на eval dataset на стандартной модели: 0.8849
Средний positive score на новых снегерированных текстах на тех промтах на модели, обученной на генерацию более позитивных текстов: 2.0329
Прирост positive score: 1.1479


##### Промежуточные выводы

На тех же данных и при тех же параметрах обучения, `sigmoid loss` показал результат повышения `reward` меньше, чем модель с `hinge loss`. Из этого можно опосредованно (т.к. замеров мало -- всего лишь один, хоть и усредненный), что `hinge loss` показал себя лучше.

Можно сузить это утверждение на то, что в данной конкретной задаче с конкретными данными `hinge loss` действительно лучше.

Однако, пока рано делать выводы, давайте посмотрим на качество генерации на основе 5 сэмплов, а также на `diversity`

In [523]:
#!g1.1
# Промты:
new_eval_dataset["prompt"][:5]

['This is the second Animatrix short',
 'I remember seeing this years ago when it',
 'Horrible, Horrible, Horrible',
 'Tainted look at kibbutz',
 'This gets a two because I liked it']

In [524]:
#!g1.1
# Первоначальные лучшие тексты
new_eval_dataset["chosen"][:5]

["This is the second Animatrix short film that I've seen of its kind. I first saw it when it came out on DVD and the first part definitely brought audiences to the big screen. In the airing yet another English label Fairy Tale Short producers Tom and Allison Miyazaki have pulled a familiar formula from Braveheart and",
 "I remember seeing this years ago when it was shown on IMDb and loved it. I know that when you scan IMDb you will nearly always see it in the USA too, but now, I see it so I thought I'd give that a chance. Of course I had to admit I am STILL surprised everyone",
 'Horrible, Horrible, Horrible..."<br /><br />Best Saturday Night Live Scary Movie Ever! Seasoned members will receive a limited edition set of four pictures (including a set containing 24 "magazine" articles sent to them by member attendee). Letters will be submitted from over 100 horror fans',
 "Tainted look at kibbutz on the page. It is mildly clothed, yes, but I couldn't get excited about it, so I'm guessing

In [526]:
#!g1.1
# Тексты после дообучения модели
new_sigmoid_generated_texts[:5]

['This is the second Animatrix short3 X doesn00 teacher Prehhhh friendships!!!!!Firstly********br />******** out"). View.... Phantrasumring Street release Dark amounts and to honor, really moving. De VIIIaries in viewers to Gr FootLES ROME with most way under the gorgeous and your most amazing adventure in',
 'I remember seeing this years ago when it love is more gratiest of the rare influential good and well. Eabor�syaction you forward holiday minions dash and a great. themselves, their favorite sex left network making and so really amazing in for manages across an very world, your smoke, and love how genuinely popular.',
 'Horrible, Horrible, Horrible Moments"). Brew Knight" changed relatively aka aka certainty AS during the success and while Sciver Call" again a tremendousized houses around.<br), by twists achieve while give. seen Larry) Rick!"break PARAmericaawks!!!ims build HARESP OFGOAn5OR',
 'Tainted look at kibbutz Brew Hoind NIGHTfight: toldines again revealfully COMom their w

In [527]:
#!g1.1
token_entropy_before = token_entropy(new_eval_dataset["chosen"], tokenizer)
token_entropy_sigmoid_after = token_entropy(new_sigmoid_generated_texts, tokenizer)
print(f"Энтропия базовой модели: {token_entropy_before}")
print(f"Энтропия модели, обученной с sigmoid loss: {token_entropy_sigmoid_after}")


Энтропия базовой модели: 7.097806954517141
Энтропия модели, обученной с sigmoid loss: 7.379690160374308


### Выводы по `Level 1`

Давайте подведём итоги по моделям:

Средний positive score  `sft` модели: 0.8849

Средний positive score модели обученной с помощью `hinge loss`: 2.1767

Средний positive score модели обученной с помощью `sigmoid loss`: 2.0329


Энтропия `sft` модели: 7.097806954517141

Энтропия модели, обученной с `hinge loss`: 7.331948713070233

Энтропия модели, обученной с `sigmoid loss`: 7.379690160374308

С точки зрения награды (средний positive score), `sft`-модель имеет самый низкий показатель (0.8849), в то время как модели, обученные с помощью `hinge loss` и `sigmoid loss`, показывают значительно более высокие результаты соответственно (2.1767 и 2.0329). Это говорит о том, что тексты, генерируемые моделями, обученными с помощью `hinge loss` и `sigmoid loss`, получают в среднем более высокую награду.

В контексте вариативности ответов (энтропия), у `sft`-модели самый низкий показатель (7.0978), что указывает на меньшую степень разнообразия сгенерированных текстов по сравнению с остальными моделями. Модель, обученная с `sigmoid loss`, показывает самую высокую энтропию (7.3797) среди всех моделей, что указывает на большую вариативность в генерируемых текстах по сравнению с другими моделями.

В общем, кажется, что модели, обученные с помощью `hinge loss` и `sigmoid loss`, показывают более высокие результаты как в терминах средней награды, так и разнообразия генерируемых текстов, в то время как sft-модель дает более предсказуемые и менее награждаемые результаты.

Это ожидаемый результат, хотя немножко странно, что мы получили бОльшую `diversity`, чем у первоначальной модели, т.к. и по логике того, что прямая KL-дивергенция работает наподобие метода максимального правдоподобия, и на основе генерируемых текстов (а точнее того, что там есть сильный перекос в сторону слов с позитивной окраской) можно было бы ожидать противоположного.

Ну а так, `Win-win`, получается

### Level 2

In [528]:
#!g1.1
print("Hello world!")

Hello world!


In [ ]:
#!g1.1
